In [30]:
import pandas as pd
import numpy as np
import pickle
import altair as alt
import os
from sklearn.metrics import accuracy_score


In [58]:
input_file_path = os.path.join('..', "data", "processed")
tgt='label'
input_file_name = os.path.join(input_file_path, "train.pck")
input_file_name_test = os.path.join(input_file_path, "test.pck")
# output_file_name = os.path.join(output_file_path, f"models_lgbm.pck")
df = pd.read_pickle(input_file_name)

models = []
scores = []
f1_scores = []

y = df.loc[:, tgt]
X = df.drop(["well_id", tgt,'row_id'], axis=1)    
X = X.fillna(np.nanmean(X))

groups = df.loc[:, 'well_id']
wells_train = np.arange(30)
wells_test=np.arange(30,100)
X_train, y_train = X.loc[df['well_id'].isin(wells_train),:],y.loc[df['well_id'].isin(wells_train)]

X_test, y_test = X.loc[df['well_id'].isin(wells_test),:],y.loc[df['well_id'].isin(wells_test)]
groups_train = df['well_id'].loc[df['well_id'].isin(wells_train)]


In [60]:
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier,XGBRFClassifier
from baikal import Input, Model, make_step
from baikal.plot import plot_model
from baikal.steps import Concatenate
from mlxtend.classifier import StackingClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import FeatureUnion, Pipeline,make_pipeline
from sklearn.svm import SVC

class GroupedShifter( BaseEstimator, TransformerMixin ):
    #Class Constructor
    
    def __init__( self):    
        print(f'shifting')
    #Return self nothing else to do here    
    def fit( self, X, y = None ):
        return self 
    
    #Method that describes what we need this transformer to do
    def transform( self, X, y = None ):
        df = pd.DataFrame(data=X,columns =[f'F_{i}' for i in range(X.shape[1])],index=np.arange(X.shape[0]))
        df['group'] = (1+np.arange(X.shape[0])) // 1100
        grouped = df.groupby('group')
        df_new_list = []
        shifts=np.arange(-50,50,5) 
        print(shifts)
        for s in shifts:
            df_shifted = grouped.shift(s)
            df_new_list.append(df_shifted.copy().values)
        X_shifted = np.concatenate(df_new_list,axis=1)
        X_all = np.concatenate([X_shifted],axis=1)
        print(X_all.shape[1])
        return X_all
    

meta_clf = make_pipeline(GroupedShifter(),LGBMClassifier(random_state=231,n_estimators = 200,learning_rate=0.08))
sclf = StackingClassifier(classifiers=[LGBMClassifier(n_estimators= 400,
                                                      learning_rate=0.08,
                               feature_fraction=0.2,
            objective="multiclassova",
            is_unbalance = True,
            num_leaves=8,
            random_state=5,
            n_jobs=-1,), XGBClassifier(random_state=123,n_estimators=300,feature_fraction=0.2),
                                      make_pipeline(StandardScaler(),
                              SVC(C=5e0,gamma='scale',probability=True))], 
                          meta_classifier=meta_clf,use_probas=True,
                          use_features_in_secondary=False,store_train_meta_features=True,verbose=1)
sclf.fit(X_train,y_train)

#model.fit(X_train, y_train)



shifting
shifting
Fitting 3 classifiers...
Fitting classifier1: lgbmclassifier (1/3)
Fitting classifier2: xgbclassifier (2/3)
Fitting classifier3: pipeline (3/3)
[-50 -45 -40 -35 -30 -25 -20 -15 -10  -5   0   5  10  15  20  25  30  35
  40  45]
300


StackingClassifier(average_probas=False,
                   classifiers=[LGBMClassifier(boosting_type='gbdt',
                                               class_weight=None,
                                               colsample_bytree=1.0,
                                               feature_fraction=0.2,
                                               importance_type='split',
                                               is_unbalance=True,
                                               learning_rate=0.08, max_depth=-1,
                                               min_child_samples=20,
                                               min_child_weight=0.001,
                                               min_split_gain=0.0,
                                               n_estimators=400, n_jobs=-1,
                                               num_leaves=8,
                                               objective='multiclassova...
                                                

In [ ]:
sclf.predict(X_test)

In [54]:
accuracy_score(y_test,sclf.predict(X_test))

[-40 -35 -30 -25 -20 -15 -10  -5   0   5  10  15  20  25  30  35]
160


0.8823896103896104

In [33]:

LGBM = make_step(LGBMClassifier)
XGB = make_step(XGBClassifier)
x_in = Input()
y_in = Input()
lgbm_p = LGBM(function = 'predict_proba')(x_in,y_in)
xgb_p = XGB(function = 'predict_proba')(x_in,y_in)
ens_features = Concatenate()([lgbm_p,xgb_p])
model = Model(x_in,ens_features,y_in)
plot_model(model,show=True,filename='model.png',dpi=120)


array([[9.99885863e-01, 2.38862477e-05, 1.51248064e-05, ...,
        1.51248064e-05, 4.92851984e-05, 2.58412298e-05],
       [9.99894511e-01, 2.38864543e-05, 1.51249372e-05, ...,
        1.51249372e-05, 4.06358598e-05, 2.58414533e-05],
       [9.99893386e-01, 3.02683833e-05, 1.40612105e-05, ...,
        1.40612105e-05, 3.92344469e-05, 2.30496937e-05],
       ...,
       [9.99597353e-01, 7.14226223e-05, 1.84255213e-05, ...,
        1.84255213e-05, 4.82794956e-05, 2.64519110e-04],
       [9.99136569e-01, 1.54440101e-04, 3.97519752e-05, ...,
        3.97519752e-05, 1.04160163e-04, 5.65079024e-04],
       [9.98681760e-01, 2.22573183e-04, 5.87206899e-05, ...,
        5.87206899e-05, 1.68782634e-04, 8.68163490e-04]])